In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender



In [2]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np


#Load Matrix
URM_all = scipy.sparse.load_npz('URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('URM_test_matrix.npz')

#Load playlistsIDS
data_playlists= pd.read_csv('all/train.csv',low_memory = False)
userID=data_playlists.playlist_id.unique()

In [3]:
CFrecommender=ItemKNNCFRecommender(URM_train)
SLIMrecommender=SLIM_BPR_Cython(URM_train)
CBFrecommender=ItemKNNCBFRecommender(ICM_all,URM_train)
UCFrecommender=UserKNNCFRecommender(URM_train)
SLIMELASTICrecommender=SLIMElasticNetRecommender(URM_train)



UCFrecommender.fit()
CFrecommender.fit()
SLIMrecommender.fit()
SLIMELASTICrecommender.fit()
CBFrecommender.fit()

SLIM_BPR_Cython: Estimated memory required for similarity matrix of 20635 items is 1703.21 MB
Compiling in Cython
Compiled module saved in subfolder: /SLIM_BPR/Cython
Compilation Complete
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 13000 ( 63 % ), 431.22 column/sec, elapsed time 0.50 min
Similarity column 20600 ( 100 % ), 450.17 column/sec, elapsed time 0.76 min
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 20600 ( 100 % ), 2160.36 column/sec, elapsed time 0.16 min


In [4]:
result1=CFrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result1["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 24.09 seconds. Users per second: 402
Recommender MAP is= 0.08346157324603644


In [5]:
result2=UCFrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result2["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 19.39 seconds. Users per second: 499
Recommender MAP is= 0.08193761205851145


In [6]:
result3=SLIMrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result3["MAP"]))

SequentialEvaluator: Processed 8001 ( 82.69% ) in 30.36 seconds. Users per second: 264
SequentialEvaluator: Processed 9676 ( 100.00% ) in 33.09 seconds. Users per second: 292
Recommender MAP is= 0.08529002028731622


In [7]:
result3=SLIMELASTICrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result3["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 27.77 seconds. Users per second: 348
Recommender MAP is= 0.06672326978768155


In [8]:
result4=CBFrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result4["MAP"]))



SequentialEvaluator: Processed 9676 ( 100.00% ) in 13.90 seconds. Users per second: 696
Recommender MAP is= 0.040625563482324885


In [36]:
Hrecommender=ItemKNNSimilarityHybridRecommender(URM_train,CFrecommender,SLIMrecommender)
Hrecommender.fit(alpha = 0.3)
result=Hrecommender.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender.fit(alpha = x)
    result=Hrecommender.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

TypeError: 'coo_matrix' object is not subscriptable

In [4]:
Hrecommender2=ItemKNNSimilarityHybridRecommender(URM_train,CFrecommender,CBFrecommender)
Hrecommender2.fit(alpha=0.8)
result=Hrecommender2.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.94 seconds. Users per second: 1219
Recommender MAP is= 0.08526994642478279


In [11]:
Hrecommender3=ItemKNNSimilarityHybridRecommender(URM_train,SLIMrecommender,CBFrecommender)
Hrecommender3.fit(alpha = 0.8)
result=Hrecommender3.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for x in alphaList:
    print(x)
    Hrecommender3.fit(alpha = x)
    result=Hrecommender3.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Processed 1165777 ( 100.00% ) in 7.79 seconds. BPR loss is 9.46E-01. Sample per second: 149671
SLIM_BPR_Recommender: Epoch 1 of 100. Elapsed time 0.30 min
Processed 1165777 ( 100.00% ) in 6.55 seconds. BPR loss is 2.14E+00. Sample per second: 178011
SLIM_BPR_Recommender: Epoch 2 of 100. Elapsed time 0.40 min
Processed 1165777 ( 100.00% ) in 5.99 seconds. BPR loss is 2.98E+00. Sample per second: 194551
SLIM_BPR_Recommender: Epoch 3 of 100. Elapsed time 0.49 min
Processed 1165777 ( 100.00% ) in 6.38 seconds. BPR loss is 3.66E+00. Sample per second: 182642
SLIM_BPR_Recommender: Epoch 4 of 100. Elapsed time 0.58 min
Processed 1165777 ( 100.00% ) in 5.67 seconds. BPR loss is 4.26E+00. Sample per second: 205504
SLIM_BPR_Recommender: Epoch 5 of 100. Elapsed time 0.67 min
Processed 1165777 ( 100.00% ) in 6.48 seconds. BPR loss is 4.76E+00. Sample per second: 179810
SLIM_BPR_Recommender: Epoch 6 of 100. Elapsed time 0.76 min
Processed 1165777 ( 100.00% ) in 5.61 seconds. BPR loss is 5.25E+00. S

Recommender MAP is= 0.06787830203832172
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.59 seconds. Users per second: 1732
Recommender MAP is= 0.0757747717271648
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.47 seconds. Users per second: 1770
Recommender MAP is= 0.08324605203035389
0.7
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.47 seconds. Users per second: 1769
Recommender MAP is= 0.08917181270675553
0.8
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.53 seconds. Users per second: 1751
Recommender MAP is= 0.09049746460294646
0.9
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.54 seconds. Users per second: 1747
Recommender MAP is= 0.08900361512058476


In [12]:
Hrecommender4=ItemKNNSimilarityHybridRecommender(URM_train,Hrecommender,Hrecommender3)
Hrecommender4.fit(alpha=0.5)
result=Hrecommender4.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender4.fit(alpha = x)
    result=Hrecommender4.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.81 seconds. Users per second: 1665
Recommender MAP is= 0.09049802733665234
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.74 seconds. Users per second: 1686
Recommender MAP is= 0.09068668414516438
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.73 seconds. Users per second: 1690
Recommender MAP is= 0.09107008948719038
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.88 seconds. Users per second: 1645
Recommender MAP is= 0.09059776431031567
0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.68 seconds. Users per second: 1704
Recommender MAP is= 0.09058166577782782
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.68 seconds. Users per second: 1704
Recommender MAP is= 0.09049802733665234
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1696
Recommender MAP is= 0.09017439987895515
0.7
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.75 seconds. Users per s

In [13]:
Hrecommender5=ItemKNNSimilarityHybridRecommender(URM_train,Hrecommender,Hrecommender2)
Hrecommender5.fit(alpha=0.5)
result=Hrecommender5.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender5.fit(alpha = x)
    result=Hrecommender5.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.72 seconds. Users per second: 1691
Recommender MAP is= 0.08635476862150258
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.65 seconds. Users per second: 1712
Recommender MAP is= 0.08570044076933668
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.68 seconds. Users per second: 1704
Recommender MAP is= 0.08620067493738416
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.69 seconds. Users per second: 1701
Recommender MAP is= 0.08662080717332794
0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.64 seconds. Users per second: 1715
Recommender MAP is= 0.08681752177583191
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.76 seconds. Users per second: 1680
Recommender MAP is= 0.08635476862150258
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.86 seconds. Users per second: 1652
Recommender MAP is= 0.08623995188827006
0.7
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.74 seconds. Users per s

In [14]:
Hrecommender6=ItemKNNSimilarityHybridRecommender(URM_train,Hrecommender2,Hrecommender3)
Hrecommender6.fit(alpha=0.5)
result=Hrecommender6.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender6.fit(alpha = x)
    result=Hrecommender6.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.79 seconds. Users per second: 1670
Recommender MAP is= 0.09037768711284642
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.79 seconds. Users per second: 1671
Recommender MAP is= 0.09079393256736179
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.83 seconds. Users per second: 1659
Recommender MAP is= 0.09111143166654524
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.79 seconds. Users per second: 1672
Recommender MAP is= 0.09085705505388947
0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.87 seconds. Users per second: 1648
Recommender MAP is= 0.09083551754341257
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.86 seconds. Users per second: 1652
Recommender MAP is= 0.09037768711284642
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.78 seconds. Users per second: 1673
Recommender MAP is= 0.08996197879251987
0.7
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.75 seconds. Users per s

In [15]:
Hrecommender7=ItemKNNSimilarityHybridRecommender(URM_train,CFrecommender,SLIMELASTICrecommender)
Hrecommender7.fit(alpha = 0.6)
result=Hrecommender7.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
#tuning
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender7.fit(alpha = x)
    result=Hrecommender7.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 14600 ( 71 % ), 483.57 column/sec, elapsed time 0.50 min
Similarity column 20600 ( 100 % ), 485.75 column/sec, elapsed time 0.71 min
Processed 5709 ( 27.67% ) in 5.00 minutes. Items per second: 19
Processed 11825 ( 57.31% ) in 10.00 minutes. Items per second: 20
Processed 18169 ( 88.05% ) in 15.00 minutes. Items per second: 20
Processed 20635 ( 100.00% ) in 16.92 minutes. Items per second: 20
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.75 seconds. Users per second: 1683
Recommender MAP is= 0.08201956775046917
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.75 seconds. Users per second: 1684
Recommender MAP is= 0.07299910103542742
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.98 seconds. Users per second: 1619
Recommender MAP is= 0.07670024623578701
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.84 seconds. Users per second: 1658
Recommender MAP is= 0.0793127111984613
0

In [16]:
Hrecommender8=ItemKNNSimilarityHybridRecommender(URM_train,CBFrecommender,SLIMELASTICrecommender)
Hrecommender8.fit(alpha = 0.4)
result=Hrecommender8.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
#tuning
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender8.fit(alpha = x)
    result=Hrecommender8.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 20600 ( 100 % ), 3252.31 column/sec, elapsed time 0.11 min
Processed 5747 ( 27.85% ) in 5.00 minutes. Items per second: 19
Processed 11553 ( 55.99% ) in 10.00 minutes. Items per second: 19
Processed 17399 ( 84.32% ) in 15.00 minutes. Items per second: 19
Processed 20635 ( 100.00% ) in 17.76 minutes. Items per second: 19
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.25 seconds. Users per second: 1845
Recommender MAP is= 0.06455190347271748
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.23 seconds. Users per second: 1849
Recommender MAP is= 0.07518462274441733
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.27 seconds. Users per second: 1835
Recommender MAP is= 0.07450483074451711
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.28 seconds. Users per second: 1832
Recommender MAP is= 0.06983443426556848
0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.31 seconds. Users

In [17]:
Hrecommender9=ItemKNNSimilarityHybridRecommender(URM_train,SLIMrecommender,SLIMELASTICrecommender)
Hrecommender9.fit(alpha = 0.5)
result=Hrecommender9.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender9.fit(alpha = x)
    result=Hrecommender9.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Processed 1165777 ( 100.00% ) in 4.76 seconds. BPR loss is 9.44E-01. Sample per second: 244813
SLIM_BPR_Recommender: Epoch 1 of 100. Elapsed time 0.19 min
Processed 1165777 ( 100.00% ) in 4.77 seconds. BPR loss is 2.14E+00. Sample per second: 244348
SLIM_BPR_Recommender: Epoch 2 of 100. Elapsed time 0.26 min
Processed 1165777 ( 100.00% ) in 5.35 seconds. BPR loss is 2.97E+00. Sample per second: 217728
SLIM_BPR_Recommender: Epoch 3 of 100. Elapsed time 0.34 min
Processed 1165777 ( 100.00% ) in 4.92 seconds. BPR loss is 3.66E+00. Sample per second: 236903
SLIM_BPR_Recommender: Epoch 4 of 100. Elapsed time 0.41 min
Processed 1165777 ( 100.00% ) in 5.50 seconds. BPR loss is 4.26E+00. Sample per second: 211817
SLIM_BPR_Recommender: Epoch 5 of 100. Elapsed time 0.49 min
Processed 1165777 ( 100.00% ) in 5.11 seconds. BPR loss is 4.77E+00. Sample per second: 228139
SLIM_BPR_Recommender: Epoch 6 of 100. Elapsed time 0.57 min
Processed 1165777 ( 100.00% ) in 4.70 seconds. BPR loss is 5.26E+00. S

SLIM_BPR_Recommender: Epoch 53 of 100. Elapsed time 4.14 min
Processed 1165777 ( 100.00% ) in 4.98 seconds. BPR loss is 1.50E+01. Sample per second: 233991
SLIM_BPR_Recommender: Epoch 54 of 100. Elapsed time 4.21 min
Processed 1165777 ( 100.00% ) in 5.56 seconds. BPR loss is 1.51E+01. Sample per second: 209490
SLIM_BPR_Recommender: Epoch 55 of 100. Elapsed time 4.29 min
Processed 1165777 ( 100.00% ) in 5.14 seconds. BPR loss is 1.52E+01. Sample per second: 226783
SLIM_BPR_Recommender: Epoch 56 of 100. Elapsed time 4.37 min
Processed 1165777 ( 100.00% ) in 4.72 seconds. BPR loss is 1.53E+01. Sample per second: 246870
SLIM_BPR_Recommender: Epoch 57 of 100. Elapsed time 4.44 min
Processed 1165777 ( 100.00% ) in 5.31 seconds. BPR loss is 1.55E+01. Sample per second: 219570
SLIM_BPR_Recommender: Epoch 58 of 100. Elapsed time 4.52 min
Processed 1165777 ( 100.00% ) in 4.96 seconds. BPR loss is 1.56E+01. Sample per second: 235248
SLIM_BPR_Recommender: Epoch 59 of 100. Elapsed time 4.60 min
Pro

Recommender MAP is= 0.08362977136926007
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.64 seconds. Users per second: 1714
Recommender MAP is= 0.08473927068722249
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.64 seconds. Users per second: 1717
Recommender MAP is= 0.08553884681714771
0.7
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.68 seconds. Users per second: 1703
Recommender MAP is= 0.08564427886162197
0.8
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.57 seconds. Users per second: 1738
Recommender MAP is= 0.08603657262305495
0.9
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.62 seconds. Users per second: 1723
Recommender MAP is= 0.0859481086321274


In [18]:
Hrecommender10=ItemKNNSimilarityHybridRecommender(URM_train,Hrecommender3,Hrecommender7)
Hrecommender10.fit(alpha=0.5)
result=Hrecommender10.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender10.fit(alpha = x)
    result=Hrecommender10.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.77 seconds. Users per second: 1676
Recommender MAP is= 0.09019968865453336
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.72 seconds. Users per second: 1692
Recommender MAP is= 0.08476138775905986
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.71 seconds. Users per second: 1694
Recommender MAP is= 0.08637173202059134
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.80 seconds. Users per second: 1668
Recommender MAP is= 0.08777400073414145
0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.85 seconds. Users per second: 1653
Recommender MAP is= 0.0894700819300434
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.68 seconds. Users per second: 1703
Recommender MAP is= 0.09019968865453336
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.84 seconds. Users per second: 1657
Recommender MAP is= 0.09057182184381402
0.7
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.78 seconds. Users per se

In [19]:
Hrecommender11=ItemKNNSimilarityHybridRecommender(URM_train,Hrecommender,Hrecommender9)
Hrecommender11.fit(alpha=0.5)
result=Hrecommender11.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender11.fit(alpha = x)
    result=Hrecommender11.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.77 seconds. Users per second: 1676
Recommender MAP is= 0.08850178099082479
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.81 seconds. Users per second: 1664
Recommender MAP is= 0.08665770545219145
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.86 seconds. Users per second: 1651
Recommender MAP is= 0.08715264416554665
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.76 seconds. Users per second: 1680
Recommender MAP is= 0.08736313387985925
0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.80 seconds. Users per second: 1668
Recommender MAP is= 0.08787813340373742
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.75 seconds. Users per second: 1683
Recommender MAP is= 0.08850178099082479
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.76 seconds. Users per second: 1679
Recommender MAP is= 0.08778560953763101
0.7
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.74 seconds. Users per s

In [20]:
Hrecommender12=ItemKNNSimilarityHybridRecommender(URM_train,Hrecommender3,Hrecommender9)
Hrecommender12.fit(alpha=0.5)
result=Hrecommender12.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender12.fit(alpha = x)
    result=Hrecommender12.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.56 seconds. Users per second: 1741
Recommender MAP is= 0.08986171306594677
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.78 seconds. Users per second: 1675
Recommender MAP is= 0.08716074591649778
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.59 seconds. Users per second: 1731
Recommender MAP is= 0.08820811486350232
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.57 seconds. Users per second: 1736
Recommender MAP is= 0.08894698988275808
0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.59 seconds. Users per second: 1730
Recommender MAP is= 0.08964733790470066
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.65 seconds. Users per second: 1713
Recommender MAP is= 0.08986171306594677
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.55 seconds. Users per second: 1744
Recommender MAP is= 0.09017133096386919
0.7
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.65 seconds. Users per s

In [21]:
Hrecommender13=ItemKNNSimilarityHybridRecommender(URM_train,Hrecommender,Hrecommender7)
Hrecommender13.fit(alpha=0.5)
result=Hrecommender13.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for x in alphaList:
    print(x)
    Hrecommender13.fit(alpha = x)
    result=Hrecommender13.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.60 seconds. Users per second: 1727
Recommender MAP is= 0.08441911514579055
0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.81 seconds. Users per second: 1664
Recommender MAP is= 0.08297291451910645
0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.74 seconds. Users per second: 1685
Recommender MAP is= 0.08316254887766512
0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.74 seconds. Users per second: 1686
Recommender MAP is= 0.08362041214982466
0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.75 seconds. Users per second: 1681
Recommender MAP is= 0.0839793355486208
0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.74 seconds. Users per second: 1685
Recommender MAP is= 0.08441911514579055
0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1698
Recommender MAP is= 0.08469652448906176
0.7
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.86 seconds. Users per se

In [22]:
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3

HrecommenderThree=ItemKNNSimilarityHybridRecommender3(URM_train,SLIMrecommender, CFrecommender,CBFrecommender)
HrecommenderThree.fit(alpha = 0.5, beta=0.3)
result=HrecommenderThree.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[(0.4,0.4),(0.6,0.2),(0.2,0.6),(0.4,0.5),(0.5,0.4),(0.6,0.3),(0.7,0.2),(0.8,0.1),(0.55,0.30),(0.6,0.3),(0.7,0.2),(0.8,0.1),(0.55,0.30)]

for a,b in alphaList:
    print(a,b)
    HrecommenderThree.fit(alpha = a, beta=b)
    result=HrecommenderThree.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Processed 1165777 ( 100.00% ) in 5.24 seconds. BPR loss is 9.44E-01. Sample per second: 222274
SLIM_BPR_Recommender: Epoch 1 of 100. Elapsed time 0.23 min
Processed 1165777 ( 100.00% ) in 4.89 seconds. BPR loss is 2.14E+00. Sample per second: 238423
SLIM_BPR_Recommender: Epoch 2 of 100. Elapsed time 0.31 min
Processed 1165777 ( 100.00% ) in 5.59 seconds. BPR loss is 2.97E+00. Sample per second: 208572
SLIM_BPR_Recommender: Epoch 3 of 100. Elapsed time 0.39 min
Processed 1165777 ( 100.00% ) in 5.28 seconds. BPR loss is 3.66E+00. Sample per second: 220925
SLIM_BPR_Recommender: Epoch 4 of 100. Elapsed time 0.46 min
Processed 1165777 ( 100.00% ) in 4.98 seconds. BPR loss is 4.25E+00. Sample per second: 234195
SLIM_BPR_Recommender: Epoch 5 of 100. Elapsed time 0.54 min
Processed 1165777 ( 100.00% ) in 4.87 seconds. BPR loss is 4.78E+00. Sample per second: 239442
SLIM_BPR_Recommender: Epoch 6 of 100. Elapsed time 0.61 min
Processed 1165777 ( 100.00% ) in 5.55 seconds. BPR loss is 5.25E+00. S

SLIM_BPR_Recommender: Epoch 53 of 100. Elapsed time 4.24 min
Processed 1165777 ( 100.00% ) in 4.78 seconds. BPR loss is 1.51E+01. Sample per second: 244089
SLIM_BPR_Recommender: Epoch 54 of 100. Elapsed time 4.32 min
Processed 1165777 ( 100.00% ) in 5.49 seconds. BPR loss is 1.51E+01. Sample per second: 212530
SLIM_BPR_Recommender: Epoch 55 of 100. Elapsed time 4.40 min
Processed 1165777 ( 100.00% ) in 5.19 seconds. BPR loss is 1.52E+01. Sample per second: 224570
SLIM_BPR_Recommender: Epoch 56 of 100. Elapsed time 4.48 min
Processed 1165777 ( 100.00% ) in 4.89 seconds. BPR loss is 1.53E+01. Sample per second: 238179
SLIM_BPR_Recommender: Epoch 57 of 100. Elapsed time 4.56 min
Processed 1165777 ( 100.00% ) in 5.60 seconds. BPR loss is 1.54E+01. Sample per second: 208346
SLIM_BPR_Recommender: Epoch 58 of 100. Elapsed time 4.64 min
Processed 1165777 ( 100.00% ) in 5.30 seconds. BPR loss is 1.56E+01. Sample per second: 220160
SLIM_BPR_Recommender: Epoch 59 of 100. Elapsed time 4.71 min
Pro

Recommender MAP is= 0.08829021982359488
0.4 0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.60 seconds. Users per second: 1727
Recommender MAP is= 0.09044649331070459
0.5 0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.57 seconds. Users per second: 1738
Recommender MAP is= 0.09114076983663473
0.6 0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.65 seconds. Users per second: 1712
Recommender MAP is= 0.09113564692236566
0.7 0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.67 seconds. Users per second: 1707
Recommender MAP is= 0.09110138707006989
0.8 0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.61 seconds. Users per second: 1726
Recommender MAP is= 0.09039001626690629
0.55 0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.67 seconds. Users per second: 1706
Recommender MAP is= 0.09174389863734148
0.6 0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.61 seconds. Users per second: 1724
Recommender MAP is= 0.09113564692236566
0.7 0.2

In [23]:
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3

HrecommenderThree2=ItemKNNSimilarityHybridRecommender3(URM_train,SLIMrecommender, CFrecommender, SLIMELASTICrecommender)
HrecommenderThree2.fit(alpha = 0.5, beta=0.3)
result=HrecommenderThree2.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[(0.4,0.4),(0.6,0.2),(0.2,0.6),(0.4,0.5),(0.5,0.4),(0.6,0.3),(0.7,0.2),(0.8,0.1),(0.55,0.30),(0.6,0.3),(0.7,0.2),(0.8,0.1),(0.55,0.30)]

for a,b in alphaList:
    print(a,b)
    HrecommenderThree2.fit(alpha = a, beta=b)
    result=HrecommenderThree2.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Processed 1165777 ( 100.00% ) in 5.66 seconds. BPR loss is 9.46E-01. Sample per second: 206123
SLIM_BPR_Recommender: Epoch 1 of 100. Elapsed time 0.22 min
Processed 1165777 ( 100.00% ) in 5.30 seconds. BPR loss is 2.14E+00. Sample per second: 219970
SLIM_BPR_Recommender: Epoch 2 of 100. Elapsed time 0.29 min
Processed 1165777 ( 100.00% ) in 4.86 seconds. BPR loss is 2.98E+00. Sample per second: 239667
SLIM_BPR_Recommender: Epoch 3 of 100. Elapsed time 0.37 min
Processed 1165777 ( 100.00% ) in 5.52 seconds. BPR loss is 3.65E+00. Sample per second: 211130
SLIM_BPR_Recommender: Epoch 4 of 100. Elapsed time 0.45 min
Processed 1165777 ( 100.00% ) in 5.19 seconds. BPR loss is 4.26E+00. Sample per second: 224717
SLIM_BPR_Recommender: Epoch 5 of 100. Elapsed time 0.53 min
Processed 1165777 ( 100.00% ) in 4.86 seconds. BPR loss is 4.80E+00. Sample per second: 239935
SLIM_BPR_Recommender: Epoch 6 of 100. Elapsed time 0.60 min
Processed 1165777 ( 100.00% ) in 5.56 seconds. BPR loss is 5.27E+00. S

SLIM_BPR_Recommender: Epoch 53 of 100. Elapsed time 4.28 min
Processed 1165777 ( 100.00% ) in 5.30 seconds. BPR loss is 1.50E+01. Sample per second: 219778
SLIM_BPR_Recommender: Epoch 54 of 100. Elapsed time 4.36 min
Processed 1165777 ( 100.00% ) in 5.05 seconds. BPR loss is 1.51E+01. Sample per second: 230893
SLIM_BPR_Recommender: Epoch 55 of 100. Elapsed time 4.44 min
Processed 1165777 ( 100.00% ) in 4.73 seconds. BPR loss is 1.52E+01. Sample per second: 246646
SLIM_BPR_Recommender: Epoch 56 of 100. Elapsed time 4.52 min
Processed 1165777 ( 100.00% ) in 5.40 seconds. BPR loss is 1.54E+01. Sample per second: 215961
SLIM_BPR_Recommender: Epoch 57 of 100. Elapsed time 4.60 min
Processed 1165777 ( 100.00% ) in 5.08 seconds. BPR loss is 1.54E+01. Sample per second: 229623
SLIM_BPR_Recommender: Epoch 58 of 100. Elapsed time 4.67 min
Processed 1165777 ( 100.00% ) in 4.35 seconds. BPR loss is 1.56E+01. Sample per second: 267974
SLIM_BPR_Recommender: Epoch 59 of 100. Elapsed time 4.74 min
Pro

Recommender MAP is= 0.08764440751343795
0.2 0.6
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.85 seconds. Users per second: 1653
Recommender MAP is= 0.08556242726026451
0.4 0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.69 seconds. Users per second: 1701
Recommender MAP is= 0.0875077518040806
0.5 0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1699
Recommender MAP is= 0.08820916971660597
0.6 0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.71 seconds. Users per second: 1696
Recommender MAP is= 0.08788266650381936
0.7 0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.80 seconds. Users per second: 1669
Recommender MAP is= 0.08781600340708914
0.8 0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.73 seconds. Users per second: 1688
Recommender MAP is= 0.0861949330127835
0.55 0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.66 seconds. Users per second: 1711
Recommender MAP is= 0.08794608545977615
0.6 0.3
S

In [24]:
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3

HrecommenderThree3=ItemKNNSimilarityHybridRecommender3(URM_train,SLIMrecommender, SLIMELASTICrecommender,CBFrecommender)
HrecommenderThree3.fit(alpha = 0.5, beta=0.3)
result=HrecommenderThree3.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[(0.4,0.4),(0.6,0.2),(0.2,0.6),(0.4,0.5),(0.5,0.4),(0.6,0.3),(0.7,0.2),(0.8,0.1),(0.55,0.30),(0.6,0.3),(0.7,0.2),(0.8,0.1),(0.55,0.30)]

for a,b in alphaList:
    print(a,b)
    HrecommenderThree3.fit(alpha = a, beta=b)
    result=HrecommenderThree3.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Processed 1165777 ( 100.00% ) in 4.68 seconds. BPR loss is 9.51E-01. Sample per second: 249015
SLIM_BPR_Recommender: Epoch 1 of 100. Elapsed time 0.20 min
Processed 1165777 ( 100.00% ) in 4.33 seconds. BPR loss is 2.15E+00. Sample per second: 269368
SLIM_BPR_Recommender: Epoch 2 of 100. Elapsed time 0.26 min
Processed 1165777 ( 100.00% ) in 4.31 seconds. BPR loss is 2.99E+00. Sample per second: 270251
SLIM_BPR_Recommender: Epoch 3 of 100. Elapsed time 0.33 min
Processed 1165777 ( 100.00% ) in 4.55 seconds. BPR loss is 3.67E+00. Sample per second: 256265
SLIM_BPR_Recommender: Epoch 4 of 100. Elapsed time 0.40 min
Processed 1165777 ( 100.00% ) in 5.26 seconds. BPR loss is 4.27E+00. Sample per second: 221742
SLIM_BPR_Recommender: Epoch 5 of 100. Elapsed time 0.48 min
Processed 1165777 ( 100.00% ) in 4.96 seconds. BPR loss is 4.79E+00. Sample per second: 235184
SLIM_BPR_Recommender: Epoch 6 of 100. Elapsed time 0.56 min
Processed 1165777 ( 100.00% ) in 5.66 seconds. BPR loss is 5.26E+00. S

SLIM_BPR_Recommender: Epoch 53 of 100. Elapsed time 3.99 min
Processed 1165777 ( 100.00% ) in 5.35 seconds. BPR loss is 1.50E+01. Sample per second: 217834
SLIM_BPR_Recommender: Epoch 54 of 100. Elapsed time 4.06 min
Processed 1165777 ( 100.00% ) in 5.06 seconds. BPR loss is 1.51E+01. Sample per second: 230297
SLIM_BPR_Recommender: Epoch 55 of 100. Elapsed time 4.14 min
Processed 1165777 ( 100.00% ) in 4.76 seconds. BPR loss is 1.52E+01. Sample per second: 244656
SLIM_BPR_Recommender: Epoch 56 of 100. Elapsed time 4.22 min
Processed 1165777 ( 100.00% ) in 5.45 seconds. BPR loss is 1.53E+01. Sample per second: 213872
SLIM_BPR_Recommender: Epoch 57 of 100. Elapsed time 4.30 min
Processed 1165777 ( 100.00% ) in 5.16 seconds. BPR loss is 1.55E+01. Sample per second: 226013
SLIM_BPR_Recommender: Epoch 58 of 100. Elapsed time 4.38 min
Processed 1165777 ( 100.00% ) in 4.91 seconds. BPR loss is 1.56E+01. Sample per second: 237603
SLIM_BPR_Recommender: Epoch 59 of 100. Elapsed time 4.46 min
Pro

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.41 seconds. Users per second: 1789
Recommender MAP is= 0.08510026069780584
0.4 0.5
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.48 seconds. Users per second: 1766
Recommender MAP is= 0.08965615414746403
0.5 0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.49 seconds. Users per second: 1763
Recommender MAP is= 0.09017601946750696
0.6 0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.52 seconds. Users per second: 1754
Recommender MAP is= 0.09022153905523593
0.7 0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.67 seconds. Users per second: 1707
Recommender MAP is= 0.09027717792233013
0.8 0.1
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.55 seconds. Users per second: 1742
Recommender MAP is= 0.089692274415814
0.55 0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.50 seconds. Users per second: 1760
Recommender MAP is= 0.09099784774619031
0.6 0.3
SequentialEvaluator: Processed 9676 ( 100.00% ) i

In [25]:
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3

HrecommenderThree4=ItemKNNSimilarityHybridRecommender3(URM_train,CFrecommender, SLIMELASTICrecommender,CBFrecommender)
HrecommenderThree4.fit(alpha = 0.5, beta=0.3)
result=HrecommenderThree4.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))
alphaList =[(0.4,0.4),(0.6,0.2),(0.2,0.6),(0.4,0.5),(0.5,0.4),(0.6,0.3),(0.7,0.2),(0.8,0.1),(0.55,0.30),(0.6,0.3),(0.7,0.2),(0.8,0.1),(0.55,0.30)]

for a,b in alphaList:
    print(a,b)
    HrecommenderThree4.fit(alpha = a, beta=b)
    result=HrecommenderThree4.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 15300 ( 74 % ), 509.55 column/sec, elapsed time 0.50 min
Similarity column 20600 ( 100 % ), 531.87 column/sec, elapsed time 0.65 min
Processed 6272 ( 30.39% ) in 5.00 minutes. Items per second: 21
Processed 12701 ( 61.55% ) in 10.00 minutes. Items per second: 21
Processed 19121 ( 92.66% ) in 15.00 minutes. Items per second: 21
Processed 20635 ( 100.00% ) in 16.26 minutes. Items per second: 21
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 20600 ( 100 % ), 2266.59 column/sec, elapsed time 0.15 min
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.04 seconds. Users per second: 1603
Recommender MAP is= 0.08551487616447362
0.4 0.4
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.96 seconds. Users per second: 1623
Recommender MAP is= 0.08485785897964267
0.6 0.2
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.68 seconds. Users per second: 1704
Recommender MAP is= 0.085

In [28]:
import random 


for y in range (600):
    a=random.uniform(0.1, 0.997)
    b=random.uniform(a, 0.997)
    HrecommenderThree.fit(alpha = a, beta=b)
    HrecommenderThree2.fit(alpha = a, beta=b)
    HrecommenderThree3.fit(alpha = a, beta=b)
    HrecommenderThree4.fit(alpha = a, beta=b)
    print(a)
    result=HrecommenderThree.evaluateRecommendations(URM_test)
    print("Recommender MAP is= {}".format(result["MAP"]))
    result2=HrecommenderThree2.evaluateRecommendations(URM_test)
    print("Recommender2 MAP is= {}".format(result2["MAP"]))
    result3=HrecommenderThree3.evaluateRecommendations(URM_test)
    print("Recommender3 MAP is= {}".format(result3["MAP"]))
    result4=HrecommenderThree4.evaluateRecommendations(URM_test)
    print("Recommender4 MAP is= {}".format(result4["MAP"]))






0.48252748874799956
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.23 seconds. Users per second: 1554
Recommender MAP is= 0.08184058804783668
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.15 seconds. Users per second: 1572
Recommender2 MAP is= 0.08746739453076872
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.76 seconds. Users per second: 1680
Recommender3 MAP is= 0.07374735501487821
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.63 seconds. Users per second: 1719
Recommender4 MAP is= 0.07067083462616573
0.4668220307533816
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.83 seconds. Users per second: 1659
Recommender MAP is= 0.08475805482555429
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.49 seconds. Users per second: 1763
Recommender2 MAP is= 0.08749802812107492
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.54 seconds. Users per second: 1746
Recommender3 MAP is= 0.07839305713807125
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.64 seco

Recommender2 MAP is= 0.08361425902178435
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.35 seconds. Users per second: 1810
Recommender3 MAP is= 0.062421172185017686
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.56 seconds. Users per second: 1741
Recommender4 MAP is= 0.06288245912006046
0.9407562642917056
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1698
Recommender MAP is= 0.06856662577318669
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.52 seconds. Users per second: 1752
Recommender2 MAP is= 0.08616251988983244
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.35 seconds. Users per second: 1808
Recommender3 MAP is= 0.06409440235317408
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.56 seconds. Users per second: 1742
Recommender4 MAP is= 0.06086306073652015
0.8723724281407579
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.79 seconds. Users per second: 1670
Recommender MAP is= 0.06874527383756762
SequentialEvaluator

Recommender4 MAP is= 0.06113870732597836
0.4550793713328001
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1698
Recommender MAP is= 0.08629539821379244
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.62 seconds. Users per second: 1722
Recommender2 MAP is= 0.08761574707145424
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.42 seconds. Users per second: 1786
Recommender3 MAP is= 0.0819422729283408
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.68 seconds. Users per second: 1704
Recommender4 MAP is= 0.07817365163093247
0.8829762125772027
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.72 seconds. Users per second: 1691
Recommender MAP is= 0.0687048093489841
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.62 seconds. Users per second: 1721
Recommender2 MAP is= 0.08591374517746096
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.36 seconds. Users per second: 1804
Recommender3 MAP is= 0.06389968355285705
SequentialEvaluator: P

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.57 seconds. Users per second: 1738
Recommender2 MAP is= 0.08722551762712812
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.42 seconds. Users per second: 1787
Recommender3 MAP is= 0.06689915783999202
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.61 seconds. Users per second: 1725
Recommender4 MAP is= 0.0632437847235566
0.7155087444746515
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.68 seconds. Users per second: 1704
Recommender MAP is= 0.07005744408062846
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.51 seconds. Users per second: 1757
Recommender2 MAP is= 0.08639662287258076
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.23 seconds. Users per second: 1852
Recommender3 MAP is= 0.06451922236041076
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.41 seconds. Users per second: 1787
Recommender4 MAP is= 0.06150873352686486
0.29230263303011084
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.81 seco

Recommender3 MAP is= 0.07598099224589482
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.59 seconds. Users per second: 1732
Recommender4 MAP is= 0.07335723634990099
0.32045967550489507
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1698
Recommender MAP is= 0.07547539329451851
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.57 seconds. Users per second: 1736
Recommender2 MAP is= 0.08610497219055892
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.37 seconds. Users per second: 1803
Recommender3 MAP is= 0.06455968893210817
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.59 seconds. Users per second: 1730
Recommender4 MAP is= 0.06269414065482262
0.9397471172866517
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.65 seconds. Users per second: 1714
Recommender MAP is= 0.06857460520629012
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.44 seconds. Users per second: 1780
Recommender2 MAP is= 0.08617544562819998
SequentialEvaluator

...

Recommender2 MAP is= 0.085887808162859
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.21 seconds. Users per second: 1858
Recommender3 MAP is= 0.0640132744711729
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.42 seconds. Users per second: 1786
Recommender4 MAP is= 0.06077510235640788
0.9302635311372998
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.55 seconds. Users per second: 1742
Recommender MAP is= 0.06888687279261525
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.46 seconds. Users per second: 1772
Recommender2 MAP is= 0.08626713512579554
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.23 seconds. Users per second: 1851
Recommender3 MAP is= 0.06424162125673538
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.43 seconds. Users per second: 1782
Recommender4 MAP is= 0.06093634047042004
0.34478202135169167
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.59 seconds. Users per second: 1732
Recommender MAP is= 0.07861175349137706
SequentialEvaluator: P

Recommender4 MAP is= 0.06118016757558124
0.2566537045422964
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.66 seconds. Users per second: 1709
Recommender MAP is= 0.08872093164199037
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.55 seconds. Users per second: 1744
Recommender2 MAP is= 0.0863761973869209
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.39 seconds. Users per second: 1795
Recommender3 MAP is= 0.0872632485331495
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.67 seconds. Users per second: 1708
Recommender4 MAP is= 0.08353877320510848
0.24084108524050485
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.53 seconds. Users per second: 1750
Recommender MAP is= 0.08928497146029743
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.54 seconds. Users per second: 1747
Recommender2 MAP is= 0.08609962120786027
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.25 seconds. Users per second: 1842
Recommender3 MAP is= 0.08697061699739221
SequentialEvaluator: 

Recommender2 MAP is= 0.08550040283518075
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.72 seconds. Users per second: 1692
Recommender3 MAP is= 0.0834523824170706
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.57 seconds. Users per second: 1737
Recommender4 MAP is= 0.08074435035409303
0.6107421159790444
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.66 seconds. Users per second: 1710
Recommender MAP is= 0.07245692231005671
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.50 seconds. Users per second: 1758
Recommender2 MAP is= 0.08692665276344337
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.44 seconds. Users per second: 1779
Recommender3 MAP is= 0.06532075704058665
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.66 seconds. Users per second: 1708
Recommender4 MAP is= 0.062415026022389596
0.7235398404947228
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.73 seconds. Users per second: 1689
Recommender MAP is= 0.07178740383760093
SequentialEvaluator:

Recommender4 MAP is= 0.061701946365986665
0.7127512130459425
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.07 seconds. Users per second: 1368
Recommender MAP is= 0.06954120336259371
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.45 seconds. Users per second: 1499
Recommender2 MAP is= 0.08623484723379647
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.98 seconds. Users per second: 1619
Recommender3 MAP is= 0.06377334493731959
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.40 seconds. Users per second: 1511
Recommender4 MAP is= 0.061140898387549776
0.3396082580329086
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.87 seconds. Users per second: 1408
Recommender MAP is= 0.08639328927182774
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.39 seconds. Users per second: 1515
Recommender2 MAP is= 0.08711234608196479
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.46 seconds. Users per second: 1497
Recommender3 MAP is= 0.08114666607441151
SequentialEvaluato

KeyboardInterrupt: 

In [23]:
import random 


for y in range (800):
    a=random.uniform(0.1, 0.99)
    print(a)
    Hrecommender10.fit(alpha = a)
    Hrecommender11.fit(alpha = a)
    Hrecommender12.fit(alpha = a)
    Hrecommender13.fit(alpha = a)
    
    result10=Hrecommender10.evaluateRecommendations(URM_test)
    result11=Hrecommender11.evaluateRecommendations(URM_test)
    result12=Hrecommender12.evaluateRecommendations(URM_test)
    result13=Hrecommender13.evaluateRecommendations(URM_test)


    print("Recommender MAP is= {}".format(result10["MAP"]))
    print("Recommender MAP is= {}".format(result11["MAP"]))
    print("Recommender MAP is= {}".format(result12["MAP"]))
    print("Recommender MAP is= {}".format(result13["MAP"]))






0.17125868936095973
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.74 seconds. Users per second: 1251
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.43 seconds. Users per second: 1301
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.83 seconds. Users per second: 1416
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.35 seconds. Users per second: 1316
Recommender MAP is= 0.08606442642798239
Recommender MAP is= 0.08710085695988155
Recommender MAP is= 0.08794247249391225
Recommender MAP is= 0.08308959030134956
0.42748891024586766
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.69 seconds. Users per second: 1258
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.35 seconds. Users per second: 1316
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.18 seconds. Users per second: 1348
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.62 seconds. Users per second: 1270
Recommender MAP is= 0.08961310734619492
Recommender MAP is= 0.08810960723628482
Recommender MAP 

SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.27 seconds. Users per second: 1331
Recommender MAP is= 0.09071037230805684
Recommender MAP is= 0.08657037825560769
Recommender MAP is= 0.09053078033266485
Recommender MAP is= 0.085392487051624
0.5917577188479912
SequentialEvaluator: Processed 9676 ( 100.00% ) in 8.34 seconds. Users per second: 1161
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.34 seconds. Users per second: 1319
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.71 seconds. Users per second: 1443
SequentialEvaluator: Processed 9676 ( 100.00% ) in 8.00 seconds. Users per second: 1210
Recommender MAP is= 0.09055284068847007
Recommender MAP is= 0.08811332717330497
Recommender MAP is= 0.09020401507065233
Recommender MAP is= 0.0846404047155817
0.43994598089210135
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.61 seconds. Users per second: 1271
SequentialEvaluator: Processed 9676 ( 100.00% ) in 8.38 seconds. Users per second: 1155
SequentialEvaluator: Process

SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.76 seconds. Users per second: 1432
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.31 seconds. Users per second: 1323
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.17 seconds. Users per second: 1350
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.43 seconds. Users per second: 1504
Recommender MAP is= 0.09064027542581192
Recommender MAP is= 0.08626251425826408
Recommender MAP is= 0.09040299118629275
Recommender MAP is= 0.08556131115059139
0.20752577284843599
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.70 seconds. Users per second: 1445
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.48 seconds. Users per second: 1293
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.86 seconds. Users per second: 1651
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1699
Recommender MAP is= 0.08654142074204474
Recommender MAP is= 0.0870949292150065
Recommender MAP is= 0.088230217825497

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.54 seconds. Users per second: 1746
Recommender MAP is= 0.08641854056762875
Recommender MAP is= 0.08713081179679924
Recommender MAP is= 0.08821508883608707
Recommender MAP is= 0.0831668140534888
0.6488622344725913
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.67 seconds. Users per second: 1706
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.53 seconds. Users per second: 1751
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.35 seconds. Users per second: 1809
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.59 seconds. Users per second: 1730
Recommender MAP is= 0.09060212833640618
Recommender MAP is= 0.08767168185465059
Recommender MAP is= 0.09029573070481849
Recommender MAP is= 0.08486633320113382
0.9743510264500453
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1699
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.52 seconds. Users per second: 1754
SequentialEvaluator: Proces

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.89 seconds. Users per second: 1642
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.58 seconds. Users per second: 1733
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.38 seconds. Users per second: 1797
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.61 seconds. Users per second: 1726
Recommender MAP is= 0.08834298798064068
Recommender MAP is= 0.08760430114143719
Recommender MAP is= 0.08914392485593431
Recommender MAP is= 0.08370114127824659
0.5937376738369036
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.72 seconds. Users per second: 1693
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.74 seconds. Users per second: 1687
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.40 seconds. Users per second: 1791
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1698
Recommender MAP is= 0.09056051258560155
Recommender MAP is= 0.08799861889219274
Recommender MAP is= 0.090197073825740

...

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.61 seconds. Users per second: 1725
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.57 seconds. Users per second: 1738
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.44 seconds. Users per second: 1778
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.62 seconds. Users per second: 1722
Recommender MAP is= 0.09084675418557475
Recommender MAP is= 0.08704120902605979
Recommender MAP is= 0.0905409154306771
Recommender MAP is= 0.08512968256676302
0.7074594007456908
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.58 seconds. Users per second: 1734
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.51 seconds. Users per second: 1756
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.36 seconds. Users per second: 1806
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.55 seconds. Users per second: 1742
Recommender MAP is= 0.09054904079567747
Recommender MAP is= 0.08743673083295615
Recommender MAP is= 0.0902920640662217

SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.59 seconds. Users per second: 1731
Recommender MAP is= 0.09021758841334522
Recommender MAP is= 0.08853068753319308
Recommender MAP is= 0.08976968356509524
Recommender MAP is= 0.084325677655197
0.2576129245135348
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.63 seconds. Users per second: 1718
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.57 seconds. Users per second: 1738
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.47 seconds. Users per second: 1769
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.52 seconds. Users per second: 1754
Recommender MAP is= 0.08715236574434714
Recommender MAP is= 0.0872396685131649
Recommender MAP is= 0.08871591229754335
Recommender MAP is= 0.08350385417744037
0.9264212432852046
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.74 seconds. Users per second: 1685
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.62 seconds. Users per second: 1722
SequentialEvaluator: Processe

SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.28 seconds. Users per second: 1540
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.82 seconds. Users per second: 1661
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.20 seconds. Users per second: 1561
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.06 seconds. Users per second: 1596
Recommender MAP is= 0.0907029447048595
Recommender MAP is= 0.08764866925537855
Recommender MAP is= 0.09008743291461692
Recommender MAP is= 0.0848299602250222
0.8905971722188393
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.09 seconds. Users per second: 1590
SequentialEvaluator: Processed 9676 ( 100.00% ) in 6.02 seconds. Users per second: 1608
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.48 seconds. Users per second: 1765
SequentialEvaluator: Processed 9676 ( 100.00% ) in 5.70 seconds. Users per second: 1696
Recommender MAP is= 0.09071664733081226
Recommender MAP is= 0.0865742739879165
Recommender MAP is= 0.09053091840406167


SequentialEvaluator: Processed 9676 ( 100.00% ) in 8.61 seconds. Users per second: 1124
Recommender MAP is= 0.09016865437400284
Recommender MAP is= 0.08827989683846847
Recommender MAP is= 0.08970196598957128
Recommender MAP is= 0.08424418849939336
0.2326321457754582
SequentialEvaluator: Processed 9676 ( 100.00% ) in 8.63 seconds. Users per second: 1121
SequentialEvaluator: Processed 9676 ( 100.00% ) in 8.59 seconds. Users per second: 1126
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.26 seconds. Users per second: 1334
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.19 seconds. Users per second: 1346
Recommender MAP is= 0.08697880690637691
Recommender MAP is= 0.08724007437862517
Recommender MAP is= 0.08846443581892031
Recommender MAP is= 0.0833156742392808
0.8472218631822767
SequentialEvaluator: Processed 9676 ( 100.00% ) in 7.86 seconds. Users per second: 1231
SequentialEvaluator: Processed 9676 ( 100.00% ) in 8.57 seconds. Users per second: 1130
SequentialEvaluator: Proces

KeyboardInterrupt: 

import re

results_run_string
m = re.search('MAP: (.+?),', results_run_string)
if m:
    found = m.group(1)
found

#BLOCCO PER FARE SUBMISSIONS
from createSubmission import create_submission

recommender=ItemKNNSimilarityHybridRecommender3(URM_all,SLIMrecommender, CFrecommender,CBFrecommender)
recommender.fit(alpha = 0.6, beta=0.2, gamma=0.2)

create_submission(recommender)

#BLOCCO PER TUNING MANUALE
import re
import pandas as pd



#Tuning Normalized
knn = [50, 100, 200,300,400, 800]  
learningRate = [0.1, 0.01, 0.025, 0.05, 0.075, 0.001]
iteration=0

for toppK in knn:
    for lr in learningRate:
        print("X"*70)
        print("Iteration  "+str(iteration))
        iteration=iteration+1
        print("Recommender KNN is= {}, LearningRate is= {}".format(toppK, lr))
        recommender.fit(epochs=30,topK = toppK, learning_rate=lr )
        results_run, results_run_string = evaluator.evaluateRecommender(recommender)
        m = re.search('MAP: (.+?),', results_run_string)
        if m:
            found = m.group(1)
        resultDf.at[toppK, lr]=found
        print("-----MAP------")
        print(found)

#Creazione del CSV
resultDf